In [34]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchvision.models import resnet101, ResNet101_Weights
import numpy as np
from scipy import signal
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# ResNet-101 + STFT

In [ ]:
fs = 1024
N = 10*fs
nperseg = 512
amp = 2 * np.sqrt(2)
noise_power = 0.001 * fs / 2
time = np.arange(N) / float(fs)
carrier = amp * np.sin(2*np.pi*50*time)
noise = np.random.normal(scale=np.sqrt(noise_power),
                         size=time.shape)
x = carrier + noise
#Compute and plot the STFT’s magnitude.
#STFTの振幅を計算してプロットします

f, t, Zxx = signal.stft(x, fs=fs, nperseg=nperseg)
plt.figure()
plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=amp)
plt.ylim([f[1], f[-1]])
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.yscale('log')
# plt.show()

In [34]:
f.shape, t.shape, Zxx.shape

((4001,), (3,), (4001, 3))

In [45]:
# init model
weights = ResNet101_Weights.IMAGENET1K_V2
model = resnet101(weights=weights)


In [47]:
num_classes = 5
batch_size = 4
epoch = 30
learning_rate = 0.01

npersegを大きくすると周波数の分解能が上がるが時間の分解能が下がる．
noverlapを大きくすると時間の分解能が上がる？

In [ ]:
radar_frame = pd.read_csv('data/radar_09.csv')
wave = radar_frame.to_numpy()
wave = wave.flatten()

scaler = StandardScaler()
wave_std = scaler.fit_transform(wave.reshape(-1, 1))
wave_std = wave_std.flatten()

fs = 2000
k = 1
nperseg = 1667 * k
noverlap = 1667 * (k - 1) + 1200

i = 12
y_wave = wave_std[1667 * i:1667 * (i + 1)]
plt.figure()
plt.plot(np.arange(len(y_wave)), y_wave)

f, t, Zxx = signal.stft(wave_std[1667 * i:1667 * (i + 1)], fs=fs, nperseg=nperseg, noverlap=noverlap, return_onesided=True)
ax = plt.figure()

plt.pcolormesh(t, f, np.abs(Zxx), vmin=np.min(np.abs(Zxx)), vmax=np.max(np.abs(Zxx)), shading="auto")
# plt.ylim(-15 - 10, -5 + 10)
plt.ylim(0, 20)
# np.abs(Zxx)
f.shape

# 全データに対してSTFTを行う

In [2]:
save_path = "./data/img/"
scaler = StandardScaler()

fs = 2000
k = 10
nperseg = 1667 * k
noverlap = 1667 * (k - 1) + 833

for i in range(20, num_class + 1):
    csv_path = "./data/radar_%02d.csv" % i
    radar_frame = pd.read_csv(csv_path)
    wave = radar_frame.to_numpy()
    wave = wave.flatten()
    wave_std = scaler.fit_transform(wave.reshape(-1, 1))
    wave_std = wave_std.flatten()
    k = 0

    while 1667 * (k + 10) < len(wave_std):
        f, t, Zxx = signal.stft(wave_std[1667 * k:1667 * (k + 10)], fs=fs, nperseg=nperseg, noverlap=noverlap, return_onesided=True)
        plt.figure()
        plt.pcolormesh(t, f, np.abs(Zxx), vmin=np.min(np.abs(Zxx)), vmax=np.max(np.abs(Zxx)), shading="auto")
        plt.ylim(0, 10)
        img_name = "stft_%02d_%03d.png" % (i, k)
        plt.savefig(save_path + img_name)
        plt.close()
        k += 10 # 10ずつずらす

In [48]:
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [17]:
img = Image.open("./data/img/stft_01_000.png").convert('RGB')
inputs = transform(img)
inputs = inputs.unsqueeze(0).to(device)

In [18]:
outputs = model(inputs)

In [25]:
inputs.size()

torch.Size([1, 3, 224, 224])

In [49]:
# image_sizeやmean, stdはデータに合わせて設定してください。
image_size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# trainデータとvalidationデータが入っているディレクトリのパスを指定
train_image_dir = './data/img/train'
val_image_dir = './data/img/val'

# trainデータ向けとvalidationデータ向けに、transformを用意します。
# 皆さんのやりたいことに合わせて適宜変更してください。
data_transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(
            image_size, scale=(0.5, 1.0)
        ),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=[-15, 15]),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
        transforms.RandomErasing(0.5),
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

# torchvision.datasets.ImageFolderでデータの入っているディレクトリのパスと
# transformを指定してあげるだけ。
train_dataset = torchvision.datasets.ImageFolder(root=train_image_dir, transform=data_transform['train'])
val_dataset = torchvision.datasets.ImageFolder(root=val_image_dir, transform=data_transform['val'])

# Datasetができたら、dataloaderに渡してあげればOK
batch_size = 8
train_dataLoader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataLoader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

# 以降はモデルを設定して学習・・・（割愛）

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [51]:
# 記録用リストの初期化

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

In [52]:
for i in range(epoch):
  train_loss = 0
  train_acc = 0
  val_loss = 0
  val_acc = 0

  #学習
  model.train()

  for images, labels in train_dataLoader:

    #勾配の初期化(ループの頭でやる必要あり)
    optimizer.zero_grad()

    # 訓練データの準備
    images = images.to(device)
    labels = labels.to(device)

    # 順伝搬計算
    outputs = model(images)

    # 誤差計算
    loss = criterion(outputs, labels)
    train_loss += loss.item()

    # 学習
    loss.backward()
    optimizer.step()

    #予測値算出
    predicted = outputs.max(1)[1]

    #正解件数算出
    train_acc += (predicted == labels).sum()

  # 訓練データに対する損失と精度の計算
  avg_train_loss = train_loss / len(train_dataLoader.dataset)
  avg_train_acc = train_acc / len(train_dataLoader.dataset)

  #評価
  model.eval()
  with torch.no_grad():

    for images, labels in val_dataLoader:

      # テストデータの準備
      images = images.to(device)
      labels = labels.to(device)

      # 順伝搬計算
      outputs = model(images)

      # 誤差計算
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      #予測値算出
      predicted = outputs.max(1)[1]

      #正解件数算出
      val_acc += (predicted == labels).sum()

    # 検証データに対する損失と精度の計算
    avg_val_loss = val_loss / len(val_dataLoader.dataset)
    avg_val_acc = val_acc / len(val_dataLoader.dataset)

  print (f'Epoch [{(i+1)}/{epoch}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
  train_loss_list.append(avg_train_loss)
  train_acc_list.append(avg_train_acc)
  val_loss_list.append(avg_val_loss)
  val_acc_list.append(avg_val_acc)

Epoch [1/30], loss: 0.25838 acc: 0.20345 val_loss: 1.12323, val_acc: 0.19178
Epoch [2/30], loss: 0.23657 acc: 0.23103 val_loss: 0.59434, val_acc: 0.21918
Epoch [3/30], loss: 0.23634 acc: 0.15862 val_loss: 0.36101, val_acc: 0.20548
Epoch [4/30], loss: 0.21948 acc: 0.21034 val_loss: 0.92598, val_acc: 0.21918
Epoch [5/30], loss: 0.22153 acc: 0.18276 val_loss: 0.22009, val_acc: 0.20548
Epoch [6/30], loss: 0.21830 acc: 0.19310 val_loss: 0.22785, val_acc: 0.20548
Epoch [7/30], loss: 0.22149 acc: 0.16897 val_loss: 0.22016, val_acc: 0.19178
Epoch [8/30], loss: 0.22074 acc: 0.21379 val_loss: 0.23106, val_acc: 0.20548
Epoch [9/30], loss: 0.21958 acc: 0.17586 val_loss: 0.22001, val_acc: 0.21918
Epoch [10/30], loss: 0.21961 acc: 0.17586 val_loss: 0.21990, val_acc: 0.20548
Epoch [11/30], loss: 0.21874 acc: 0.18966 val_loss: 0.22034, val_acc: 0.19178
Epoch [12/30], loss: 0.22534 acc: 0.17586 val_loss: 0.21960, val_acc: 0.20548
Epoch [13/30], loss: 0.21945 acc: 0.20000 val_loss: 0.22129, val_acc: 0.1